In [ ]:
            
import os
import pandas as pd
import openpyxl
import io
import msoffcrypto

def process_excel_file(file_path, password=None, sheet_name=None, skiprow=None, dtype=None):
    try:
        if password is not None:
            with open(file_path, 'rb') as file:
                office_file = msoffcrypto.OfficeFile(file)
                office_file.load_key(password=password)
                decrypted_workbook = io.BytesIO()
                office_file.decrypt(decrypted_workbook)
                print("Decryption successful")

            data = pd.read_excel(decrypted_workbook, sheet_name=sheet_name, skiprows=skiprow, dtype=dtype)
            return data
        else:
            data = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprow, dtype=dtype)
            return data
    except Exception as e:
        print(f"Error processing Excel file: {e}")
        return None

def process_files_recursive(directory_path, file_name):
    try:
        for item_name in os.listdir(directory_path):
            item_path = os.path.join(directory_path, item_name)

            if os.path.isdir(item_path):
                # If the current item is a directory, recurse into it
                result = process_files_recursive(item_path, file_name)
                if result:
                    return result
            elif item_name == file_name:
                # If the current item is the specific Excel file, return its path
                return item_path
    except Exception as e:
        print(f"Error processing files recursively: {e}")
        return None

def calculate_sum_conditions(data):
    try:
        # Assuming data is a DataFrame with columns 'Factory' and 'PTAX'

        # AP conditions
        AP_conditions = data['Factory'].isin(['M44'])
        AP = data.loc[AP_conditions, 'PTAX'].sum()

        # TN conditions
        TN_conditions = data['Factory'].isin(['L27', 'M38', '38M'])
        TN = data.loc[TN_conditions, 'PTAX'].sum()
        
        # Karnataka conditions
        Karnataka_conditions = ~(AP_conditions | TN_conditions)
        Karnataka = data.loc[Karnataka_conditions, 'PTAX'].sum()

        # Total sum
        total_sum = data['PTAX'].sum()

        # Create a DataFrame with the results
        result_df = pd.DataFrame({
            'AP': [AP],
            'TN': [TN],
            'Karnataka': [Karnataka],
            'Total': [total_sum]
        })

        return result_df
    except Exception as e:
        print(f"Error calculating sum conditions: {e}")
        return None

def paysheet_total_ptax(main_path, file_name, sheet_name, skiprows=None, password=None):
    try:
        file_path = process_files_recursive(main_path, file_name)

        if file_path:
            print(f"File found at: {file_path}")
            table = process_excel_file(file_path, sheet_name=sheet_name, skiprow=skiprows, password = password)
            if table is not None:
                result_dataframe = calculate_sum_conditions(table)
                return result_dataframe
            else:
                print("Error processing Excel file.")
        else:
            print(f"File not found: {file_name}")
            return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

main_path = "/home/finstein-emp/Desktop/Post Disbursement/Sep"
file_name1 = "Staff left and live paysheet Sept 2023 After locking.xlsx"
sheet_name1 = "audit_summary_staff"  # Specify the desired sheet name
skiprows = 2
password = "shahipayroll"
total_ptax_staffs = paysheet_total_ptax(main_path, file_name1, sheet_name1, skiprows, password)
# Example usage:
file_name2 = "Consolidated Audit Summary Sep-23 All.xlsx"
sheet_name2 = "Paysheet"  # Specify the desired sheet name

total_ptax_workers = paysheet_total_ptax(main_path, file_name2, sheet_name2)

total_ptax_multiple = pd.concat([total_ptax_staffs, total_ptax_workers], ignore_index=True)
total_ptax = total_ptax_multiple.sum(axis = 0)
ptax_dataframe = pd.DataFrame(total_ptax).transpose()
ptax_dataframe